In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scripts.data_import import data_import
from scripts.run_optimization_model import run_model

# Read Data

In [2]:
file = 'data'
path = 'data/' + file + '.xlsx'
data, basis, bess, service, revenue_change = data_import(path)
bess['s_eff'] = 1
# bess['cap_power'] = 1000
# bess['cap_energy'] = 1000

In [3]:
service['arb'] = 0
service['reg'] = 0
service['dr'] = 0
service['il'] = 0
service['pres'] = 0
service['cres'] = 0
service['reg_symmetric'] = 1
service['ec'] = 1
service['load'] = 1

data['schedule']['ec'] = 1

In [4]:
packaged_data = [data, basis, bess, service] 

annual_revenue, result_df, revenue_df, financial_metrics = run_model(packaged_data, result_html_name = [])

In [5]:
annual_revenue

261115.73536149022

In [ ]:
revenue_df

In [ ]:
result_df[result_df.storage_to_load != 0]

In [ ]:
annual_revenue

In [ ]:
df

# RUN

In [ ]:
power_range = list(range(1, 1))
energy_range = list(range(1, 1))

econs_sens_keys = []
econs_sens_values = []

for power in power_range:
    for energy in energy_range:

        power_target = power * 0.5
        energy_target = energy * 0.5

        print("Solving: ", power_target, energy_target)

        bess['cap_power'] = power_target
        bess['cap_energy'] = energy_target

        packaged_data = [data, basis, bess, service] 

        annual_revenue, result_df, revenue_df, financial_metrics = run_model(packaged_data, result_html_name = [])

        [NPV, ROI, IRR, BCR, PP] = financial_metrics

        econs_sens_keys.append([power_target, energy_target])
        econs_sens_values.append([annual_revenue] + financial_metrics)


In [6]:
df = pd.DataFrame(index = pd.MultiIndex.from_tuples(econs_sens_keys, names = ['Power', 'Energy']), data = econs_sens_values, columns = ['annual_revenue', 'NPV', 'ROI', 'IRR', 'BCR', 'PP'])
df['annual_revenue_per_kw'] = df['annual_revenue']/df.index.get_level_values(0)/1000
df = df.reset_index()
# df['Energy'] = (df['Energy'] * 0.5) * df['Power']
# df = df.set_index(['Power', 'Energy'])

In [ ]:
pd.set_option('display.max_rows', None)
df

In [ ]:
pivot_df = df.pivot(index="Energy", columns="Power", values="annual_revenue_per_kw") * 100
pivot_df

In [ ]:
import plotly.graph_objects as go

# Data for the heatmap
z = pivot_df.values
x = pivot_df.columns
y = pivot_df.index

# Creating the heatmap
fig = go.Figure(data=go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='RdBu',
    reversescale=True,  
    colorbar=dict(title='')
))

# Adding titles and labels
fig.update_layout(
    # title='Heatmap of ROI with varying Power and Energy (Reversed Energy Axis)',
    xaxis=dict(title='Power'),
    yaxis=dict(title='Energy')
)

# Display the figure
fig.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Calculating Energy/Power ratio for the contour plot
df['Ratio'] = df['Energy'] / df['Power']
pivot_ratio = df.pivot(index="Energy", columns="Power", values="Ratio")

# Data for the heatmap
z = pivot_df.values
x = pivot_df.columns
y = pivot_df.index

# Creating the heatmap with contour lines
fig = go.Figure()

# Add heatmap
fig.add_trace(go.Heatmap(
    z=z,
    x=x,
    y=y,
    colorscale='RdBu',
    reversescale=True,  # High values red, low values blue
    colorbar=dict(title='ROI (%)'),
    zmin=np.min(z),  # Define the range of z for better contrast
    zmax=np.max(z)
))

# Add contour lines for the Energy/Power ratio with text labels
fig.add_trace(go.Contour(
    z=pivot_ratio.values,
    x=pivot_df.columns,
    y=pivot_df.index,
    showscale=False,  # Hide the scale for contour lines
    contours=dict(
        coloring='lines',
        showlabels=True,  # Enable labels on contours
        labelfont=dict(size=12, color='black'),  # Customize the font of the labels
        start=0.5,  # Starting value for contour lines
        end=4,     # Ending value for contour lines
        size=0.5   # Interval between contour lines
    ),
    line=dict(color='black', width=2),
    # ncontours=50  # Increase the number of contour lines

))

# Adding titles and labels, and setting axis ranges to start from 1
fig.update_layout(
    title='Heatmap of ROI with Energy/Power Ratio Contour Lines',
    xaxis=dict(title='Power', range=[0.5, 8]),
    yaxis=dict(title='Energy', range=[0.5, 8])
)

# Display the figure
fig.show()
